In [1]:
!pip install geopandas
!pip install geopy
!pip install folium

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

     |████████████████████████████████| 972kB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 30.8MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 34.2MB/s eta 0:00:01     |██████████████████████████▍     | 12.2MB 34.2MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 34.4MB/s eta 0:00:01
     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [2]:
#Download a Dataset from Barcelona Open Data council
url = 'https://opendata-ajuntament.barcelona.cat/data/dataset/79bdf758-dae1-485b-800c-be9f8cfa9360/resource/e7206797-e57b-4ded-8c6c-62e9b4cb54f7/download/2017_distribucio_territorial_renda_familiar.csv'
bcn_rfd = pd.read_csv(url)

#Some treatment to clean the Data set
bcn_rfd.loc[bcn_rfd.Nom_Barri=='Vallvidrera, el Tibidabo i les Planes', 'Nom_Barri'] = 'Vallvidrera' #Shortening the neighborhood name
bcn_rfd = bcn_rfd.drop(['Any'], axis=1)
bcn_rfd = bcn_rfd.rename(columns={'Índex RFD Barcelona = 100': 'RFD', 'Població': 'Population'})
bcn_rfd.head()

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Population,RFD
0,1,Ciutat Vella,1,el Raval,47986,71.2
1,1,Ciutat Vella,2,el Barri Gòtic,16240,106.1
2,1,Ciutat Vella,3,la Barceloneta,15101,79.6
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22923,99.4
4,2,Eixample,5,el Fort Pienc,32048,106.5


In [3]:
# Getting the coordinates of each Neighborhood
locator = Nominatim(user_agent='myGeocoder')
bcn_lon = []
bcn_lat = []

for index, row in bcn_rfd.iterrows():
    try:
        location = locator.geocode(row['Nom_Barri'] + ', Barcelona')
        bcn_lon.append(location.longitude)
        bcn_lat.append(location.latitude)
    except:
        bcn_lon.append('Error')
        bcn_lat.append('Error')
        print("An exception occurred")

        #Adding the coordinates to the data frame
bcn_rfd['Latitude'] = bcn_lat
bcn_rfd['Longitude'] = bcn_lon
bcn_rfd.head()


,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Population,RFD,Latitude,Longitude
0,1,Ciutat Vella,1,el Raval,47986,71.2,41.379518,2.168368
1,1,Ciutat Vella,2,el Barri Gòtic,16240,106.1,41.383395,2.176912
2,1,Ciutat Vella,3,la Barceloneta,15101,79.6,41.380653,2.189927
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22923,99.4,41.390934,2.182200
4,2,Eixample,5,el Fort Pienc,32048,106.5,41.395925,2.182325


In [4]:
#Getting Barcelona's coordinates
bcn_latitude = locator.geocode('Barcelona').latitude
bcn_longitude = locator.geocode('Barcelona').longitude

# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[bcn_latitude, bcn_longitude], zoom_start=12)

# add markers to map
for lat, lng, barrio, distrito in zip(bcn_rfd['Latitude'], bcn_rfd['Longitude'], bcn_rfd['Nom_Barri'], bcn_rfd['Nom_Districte']):
    label = '{}, {}'.format(barrio, distrito)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
map_barcelona

# Clustering Barcelona's neigbourhoods

In [5]:
# set number of clusters
kclusters = 5

#We are going to try different clusters with different columns
#First cluster with RFD Index, Population and coordinates

bcn_clus = bcn_rfd.drop(['Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bcn_clus)

# check cluster labels generated for each row in the dataframe
clus = kmeans.labels_ 

bcn_rfd['Cluster'] = clus
bcn_rfd.head()

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Population,RFD,Latitude,Longitude,Cluster
0,1,Ciutat Vella,1,el Raval,47986,71.2,41.379518,2.168368,3
1,1,Ciutat Vella,2,el Barri Gòtic,16240,106.1,41.383395,2.176912,0
2,1,Ciutat Vella,3,la Barceloneta,15101,79.6,41.380653,2.189927,0
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22923,99.4,41.390934,2.182200,2
4,2,Eixample,5,el Fort Pienc,32048,106.5,41.395925,2.182325,2


In [6]:
#Second cluster only with RFD Index

bcn_clus2 = bcn_rfd.drop(['Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri', 'Latitude', 'Longitude', 'Population'], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bcn_clus2)

# check cluster labels generated for each row in the dataframe
clus2 = kmeans.labels_ 

bcn_rfd['Cluster2'] = clus2
bcn_rfd.head()

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Population,RFD,Latitude,Longitude,Cluster,Cluster2
0,1,Ciutat Vella,1,el Raval,47986,71.2,41.379518,2.168368,3,1
1,1,Ciutat Vella,2,el Barri Gòtic,16240,106.1,41.383395,2.176912,0,2
2,1,Ciutat Vella,3,la Barceloneta,15101,79.6,41.380653,2.189927,0,1
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22923,99.4,41.390934,2.182200,2,2
4,2,Eixample,5,el Fort Pienc,32048,106.5,41.395925,2.182325,2,2


In [7]:
#Sorting the resoults to check wich clusters fits better with our objective
bcn_rfd = bcn_rfd.sort_values('RFD', ascending=False)
bcn_rfd.head(15)

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Population,RFD,Latitude,Longitude,Cluster,Cluster2
20,4,Les Corts,21,Pedralbes,12117,248.8,41.390140,2.112218,0,3
23,5,Sarrià-Sant Gervasi,24,les Tres Torres,16660,215.8,41.397611,2.131184,0,3
22,5,Sarrià-Sant Gervasi,23,Sarrià,25106,193.6,41.399373,2.121513,2,3
25,5,Sarrià-Sant Gervasi,26,Sant Gervasi - Galvany,47753,192.1,41.397807,2.143377,3,3
24,5,Sarrià-Sant Gervasi,25,Sant Gervasi - la Bonanova,25909,184.6,41.405983,2.133405,2,3
6,2,Eixample,7,la Dreta de l'Eixample,44246,175.9,41.395037,2.167207,1,0
66,10,Sant Martí,67,la Vila Olímpica del Poblenou,9404,164.2,41.389868,2.196846,0,0
68,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,13710,150.1,41.405228,2.213352,0,0
26,5,Sarrià-Sant Gervasi,27,el Putxet i el Farró,29617,144.6,41.407476,2.143283,2,0
21,5,Sarrià-Sant Gervasi,22,Vallvidrera,4689,144.1,41.415067,2.107482,4,0


In [8]:
#Printing the clusters in the map

# create map of Barcelona using latitude and longitude values
map_bcn_clus = folium.Map(location=[bcn_latitude, bcn_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, barrio, cluster in zip(bcn_rfd['Latitude'], bcn_rfd['Longitude'], bcn_rfd['Nom_Barri'], bcn_rfd['Cluster2']):
    label = '{}, {}'.format(barrio, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_bcn_clus)
       
    
map_bcn_clus


In [11]:
#Printing the clusters in the map

# create map of Barcelona using latitude and longitude values
map_bcn_clus = folium.Map(location=[bcn_latitude, bcn_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, barrio, cluster in zip(bcn_rfd['Latitude'], bcn_rfd['Longitude'], bcn_rfd['Nom_Barri'], bcn_rfd['Cluster']):
    label = '{}, {}'.format(barrio, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_bcn_clus)
       
    
map_bcn_clus

# Working with Foursquare API

In [9]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [10]:
#Setting up the Foursquare API call method
CLIENT_ID = '0OVR2DRQB2SAWSASH5154OLEEA30BU55IU1SEXXWHTWGNA23' # your Foursquare ID
CLIENT_SECRET = '3H5WY4B5LIGBECC0WEAMSVCZYH55TQO5Z0TWXJ5GU5SYS2ZM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
search_query = 'hotel'
radius = 10000

In [11]:
#Calling the API
df = pd.DataFrame()
for lat, lon in zip(bcn_rfd['Latitude'], bcn_rfd['Longitude']):
    latitude = lat
    longitude = lon
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    hotels = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(hotels)
    
    #Adding the new dataframe to the existing one
    df = df.append(dataframe)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [12]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

#Dropping all duplicate values
df_filtered.drop_duplicates(subset ="address", keep = 'first', inplace = True) 

df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Fairmont Hotel Rey Juan Carlos I (Hotel Rey Ju...,Hotel,"Av. Diagonal, 661-671",ES,Barcelona,España,NaN,1021,"[Av. Diagonal, 661-671, 08028 Barcelona Catalu...","[{'label': 'display', 'lat': 41.38128935295143...",41.381289,2.109007,Les Corts,08028,Cataluña,4e38aa161fc7e413a51a4788
1,Hotel Sansi Pedralbes,Hotel,"Av. Pearson, 1-3",ES,Barcelona,España,NaN,369,"[Av. Pearson, 1-3, Barcelona Cataluña, España]","[{'label': 'display', 'lat': 41.39343506028381...",41.393435,2.111722,NaN,NaN,Cataluña,4bcc5f5baeaaeee1a1f23d6d
2,Piscina Hotel Rey Juan Carlos I,Hotel Pool,Hotel Rey Juan Carlos I,ES,Barcelona,España,NaN,986,"[Hotel Rey Juan Carlos I, 08028 Barcelona Cata...","[{'label': 'display', 'lat': 41.38138119681831...",41.381381,2.110405,NaN,08028,Cataluña,577e7162498eb2d9a3604b44
3,abba Garden hotel 4*,Hotel,"C/ Santa Rosa, 33",ES,Esplugues de Llobregat,España,NaN,1008,"[C/ Santa Rosa, 33, 08950 Esplugues de Llobreg...","[{'label': 'display', 'lat': 41.3853264084012,...",41.385326,2.101987,NaN,08950,Cataluña,4bc7785993bdeee11c6b37ae
4,Hotel Catalonia Rigoletto ****,Hotel,Sabino de Arana 22-24,ES,Barcelona,España,NaN,1215,"[Sabino de Arana 22-24, 08028 Barcelona Catalu...","[{'label': 'display', 'lat': 41.38559623122477...",41.385596,2.125446,NaN,08028,Cataluña,4b97eac4f964a520341f35e3
5,Hotel Sofia Barcelona - In the Unbound Collect...,Hotel,Plaça De Pius Xii 4,ES,Barcelona,España,NaN,1035,"[Plaça De Pius Xii 4, 08028 Barcelona Cataluña...","[{'label': 'display', 'lat': 41.38598, 'lng': ...",41.385980,2.123314,NaN,08028,Cataluña,5d30def44318c90008b07cba
6,AC Hotel Victoria Suites by Marriott,Hotel,"Beltran I Ròzpide, 7-9",ES,Barcelona,España,NaN,767,"[Beltran I Ròzpide, 7-9, 08034 Barcelona Catal...","[{'label': 'display', 'lat': 41.389324, 'lng':...",41.389324,2.121338,NaN,08034,Cataluña,4bc5a24b0a30d13a94385a9c
7,Hotel Catalonia Castellnou ***,Hotel,Castellnou 61,ES,Barcelona,España,NaN,1546,"[Castellnou 61, 08017 Barcelona Cataluña, España]","[{'label': 'display', 'lat': 41.39654736546470...",41.396547,2.128653,NaN,08017,Cataluña,4c10c334f1b6a59348557c27
8,Gimnasio del Gran Hotel Princesa Sofía,Gym,NaN,ES,Barcelona,España,NaN,901,"[Barcelona Cataluña, España]","[{'label': 'display', 'lat': 41.387151, 'lng':...",41.387151,2.122252,NaN,NaN,Cataluña,506c8271e4b08b16978f3328
9,Gran Hotel Princesa Sofía,Hotel,"Plaça de Pius XII, 4",ES,Barcelona,España,Av. Diagonal,992,"[Plaça de Pius XII, 4 (Av. Diagonal), 08028 Ba...","[{'label': 'display', 'lat': 41.38651744622853...",41.386517,2.123076,NaN,08028,Cataluña,4adcda4cf964a520b93f21e3


# Working with Open Data set

In [15]:
url2 = 'https://opendata-ajuntament.barcelona.cat/data/dataset/88efe464-2bcd-4794-85b0-8b0bbfd9e4c0/resource/eced0fe8-9892-4926-b035-4fdc7328e31d/download'
bcn_hotels = pd.read_csv(url2)

#Dropping all duplicate values
bcn_hotels.drop_duplicates(subset ="CODI_EQUIPAMENT", keep = 'first', inplace = True) 

bcn_hotels

,CODI_EQUIPAMENT,EQUIPAMENT,SECCIO,TIPUS_VIA,NOM_CARRER,NUM_CARRER_1,NUM_CARRER_2,CODI_BARRI,NUM_BARRI,CODI_DISTRICTE,...,HORARI_PERIODE_INICI,HORARI_PERIODE_FI,HORARI_DIES,HORARI_HORES_INICI,HORARI_HORES_FI,HORARI_OBSERVACIONS,3ER_NIVELL,2N_NIVELL,1ER_NIVELL,Unnamed: 32
0,12124527,Hotel Amrey Sant Pau - HB-004046,#,C,Sant Antoni Maria Claret,173,173.0,35,el Guinardó,7,...,NaN,NaN,NaN,NaN,NaN,NaN,Hotels 2 estr.,Hotels,Allotjament,NaN
2,75145812,Hotel Sansi Pedralbes - HB-004086,#,Av,Pearson,1,3.0,21,Pedralbes,4,...,NaN,NaN,NaN,NaN,NaN,NaN,Hotels 4 estr.,Hotels,Allotjament,NaN
4,139120246,Hotel Silken Sant Gervasi - HB-004054,#,C,Sant Gervasi de Cassoles,26,26.0,25,Sant Gervasi - la Bonanova,5,...,NaN,NaN,NaN,NaN,NaN,NaN,3 estrelles,Hotels,Allotjament,NaN
6,182121024,Hotel Attica21 Barcelona Mar - HB-004340,#,C,Provençals,10,10.0,35,el Guinardó,10,...,NaN,NaN,NaN,NaN,NaN,NaN,Hotels 4 estr.,Hotels,Allotjament,NaN
8,273135303,Abba Sants Hotel - HB-004078,#,C,Numància,32,32.0,18,Sants,3,...,NaN,NaN,NaN,NaN,NaN,NaN,4 estrelles,Hotels,Allotjament,NaN
10,1026133729,Euro Hotel Barcelona Diagonal Port - HB-004195,#,C,Lope de Vega,4,4.0,35,el Guinardó,10,...,NaN,NaN,NaN,NaN,NaN,NaN,Hotels 4 estr.,Hotels,Allotjament,NaN
12,1026134324,Hotel Garbí Mil·lenni - HB-004157,#,Rda,Sant Pau,14,14.0,1,el Raval,1,...,NaN,NaN,NaN,NaN,NaN,NaN,4 estrelles,Hotels,Allotjament,NaN
14,1044121532,Hotel Viladomat - HB-004071,#,C,Viladomat,197,197.0,9,la Nova Esquerra de l'Eixample,2,...,NaN,NaN,NaN,NaN,NaN,NaN,3 estrelles,Hotels,Allotjament,NaN
16,1051101151,Hotel Front Maritim - HB-004149,#,Pg,Garcia Fària,69,69.0,69,Diagonal Mar i el Front Marítim del Poblenou,10,...,NaN,NaN,NaN,NaN,NaN,NaN,4 estrelles,Hotels,Allotjament,NaN
18,1068120923,Hotel Barcelona Universal - HB-004114,#,Av,Paral·lel,76,80.0,1,el Raval,1,...,NaN,NaN,NaN,NaN,NaN,NaN,Hotels 4 estr.,Hotels,Allotjament,NaN


In [16]:
#Grouping the hotels by neighbourhod and cleaning and sorting the result

bcn_hotels['num_hotels'] = bcn_hotels.groupby('CODI_BARRI')['CODI_BARRI'].transform('count')
hotels_barri = bcn_hotels.filter(['CODI_BARRI', 'num_hotels'], axis=1)
hotels_barri.drop_duplicates(subset ="CODI_BARRI", keep = 'first', inplace = True)
hotels_barri = hotels_barri.sort_values('num_hotels', ascending=True)
hotels_barri = hotels_barri.rename(columns={'CODI_BARRI': 'Codi_Barri'})
hotels_barri.head()

,Codi_Barri,num_hotels
810,17,1
334,43,1
374,29,1
194,12,1
412,22,1


In [17]:
#hotels_barri.set_index("Codi_Barri",inplace=True)
#bcn_rfd.set_index("Codi_Barri",inplace=True)

bcn = bcn_rfd.merge(hotels_barri, left_index=True, right_on='Codi_Barri', how='left')
bcn.reset_index(inplace=True, drop=True)
bcn = bcn.drop(['Codi_Barri_x', 'Codi_Barri_y'], axis=1)
bcn['num_hotels'] = bcn['num_hotels'].fillna(0)
bcn.head(15)

,Codi_Barri,Codi_Districte,Nom_Districte,Nom_Barri,Population,RFD,Latitude,Longitude,Cluster,Cluster2,num_hotels
0,20,4,Les Corts,Pedralbes,12117,248.8,41.390140,2.112218,0,3,7.0
1,23,5,Sarrià-Sant Gervasi,les Tres Torres,16660,215.8,41.397611,2.131184,0,3,3.0
2,22,5,Sarrià-Sant Gervasi,Sarrià,25106,193.6,41.399373,2.121513,2,3,1.0
3,25,5,Sarrià-Sant Gervasi,Sant Gervasi - Galvany,47753,192.1,41.397807,2.143377,3,3,6.0
4,24,5,Sarrià-Sant Gervasi,Sant Gervasi - la Bonanova,25909,184.6,41.405983,2.133405,2,3,3.0
5,6,2,Eixample,la Dreta de l'Eixample,44246,175.9,41.395037,2.167207,1,0,3.0
6,66,10,Sant Martí,la Vila Olímpica del Poblenou,9404,164.2,41.389868,2.196846,0,0,11.0
7,68,10,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou,13710,150.1,41.405228,2.213352,0,0,9.0
8,26,5,Sarrià-Sant Gervasi,el Putxet i el Farró,29617,144.6,41.407476,2.143283,2,0,11.0
9,21,5,Sarrià-Sant Gervasi,Vallvidrera,4689,144.1,41.415067,2.107482,4,0,2.0


In [18]:
#Clustering neigbourhoods with the number of hotels and RFD
# set number of clusters
kclusters = 5

#We are going to try different clusters with different columns
#Third cluster with RFD Index and number of hotels
bcn_clus3 = bcn.drop(['Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri', 'Latitude', 'Longitude', 'Population', 'Latitude', 'Longitude', 'Cluster', 'Cluster2'], 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bcn_clus3)

# check cluster labels generated for each row in the dataframe
clus3 = kmeans.labels_ 

bcn['Cluster3'] = clus3
bcn



,Codi_Barri,Codi_Districte,Nom_Districte,Nom_Barri,Population,RFD,Latitude,Longitude,Cluster,Cluster2,num_hotels,Cluster3
0,20,4,Les Corts,Pedralbes,12117,248.8,41.390140,2.112218,0,3,7.0,1
1,23,5,Sarrià-Sant Gervasi,les Tres Torres,16660,215.8,41.397611,2.131184,0,3,3.0,1
2,22,5,Sarrià-Sant Gervasi,Sarrià,25106,193.6,41.399373,2.121513,2,3,1.0,1
3,25,5,Sarrià-Sant Gervasi,Sant Gervasi - Galvany,47753,192.1,41.397807,2.143377,3,3,6.0,1
4,24,5,Sarrià-Sant Gervasi,Sant Gervasi - la Bonanova,25909,184.6,41.405983,2.133405,2,3,3.0,1
5,6,2,Eixample,la Dreta de l'Eixample,44246,175.9,41.395037,2.167207,1,0,3.0,3
6,66,10,Sant Martí,la Vila Olímpica del Poblenou,9404,164.2,41.389868,2.196846,0,0,11.0,3
7,68,10,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou,13710,150.1,41.405228,2.213352,0,0,9.0,3
8,26,5,Sarrià-Sant Gervasi,el Putxet i el Farró,29617,144.6,41.407476,2.143283,2,0,11.0,3
9,21,5,Sarrià-Sant Gervasi,Vallvidrera,4689,144.1,41.415067,2.107482,4,0,2.0,3


In [19]:
#Printing the clusters in the map

# create map of Barcelona using latitude and longitude values
map_bcn_clus = folium.Map(location=[bcn_latitude, bcn_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, barrio, cluster in zip(bcn['Latitude'], bcn['Longitude'], bcn['Nom_Barri'], bcn['Cluster3']):
    label = '{}, {}'.format(barrio, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_bcn_clus)
       
    
map_bcn_clus

In [35]:
goal = bcn[bcn.Cluster3.eq(1)]
goal

,Codi_Barri,Codi_Districte,Nom_Districte,Nom_Barri,Population,RFD,Latitude,Longitude,Cluster,Cluster2,num_hotels,Cluster3
0,20,4,Les Corts,Pedralbes,12117,248.8,41.390140,2.112218,0,3,7.0,1
1,23,5,Sarrià-Sant Gervasi,les Tres Torres,16660,215.8,41.397611,2.131184,0,3,3.0,1
2,22,5,Sarrià-Sant Gervasi,Sarrià,25106,193.6,41.399373,2.121513,2,3,1.0,1
3,25,5,Sarrià-Sant Gervasi,Sant Gervasi - Galvany,47753,192.1,41.397807,2.143377,3,3,6.0,1
4,24,5,Sarrià-Sant Gervasi,Sant Gervasi - la Bonanova,25909,184.6,41.405983,2.133405,2,3,3.0,1
